In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [2]:
!pip install tensorflow

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [2]:
def preprocess_df(df):
    df["deadline"] = pd.to_datetime(df["deadline"])
    df["launched"] = pd.to_datetime(df["launched"])
    df["success"] = df["pledged"] >= df["goal"]
    df["duration"] = df["deadline"] - df["launched"]
    
    return df

In [3]:
def clean_df(df):
    df = df.drop(df.loc[df["duration"] < datetime.timedelta(days=1)].index)
    df = df.drop(df.loc[df["state"] == "live"].index)
    df = df.drop(df.loc[df["state"] == "canceled"].index)
    
    return df

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

def get_datetime_seconds(x):
    return x.seconds
tokenizer = None
def get_X_X_word_matrix_y(df, is_train=True):
    global tokenizer
    if not is_train and tokenizer is None:
        raise Exception("tokenizer not fit yet")
    
    X = df[["name", "duration", "usd_goal_real", "main_category", "category"]].copy()
    X["duration"] = X["duration"].apply(lambda x: x.seconds)
    X["main_category"] = X["main_category"].astype("category")
    X["category"] = X["category"].astype("category")
    
    mc_one_hot = pd.get_dummies(X["main_category"])
    c_one_hot = pd.get_dummies(X["category"])
    X = X.drop("main_category", axis=1)
    X = X.drop("category", axis=1)
    X = X.join(mc_one_hot, rsuffix="_maincat")
    X = X.join(c_one_hot, rsuffix="_cat")
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X["name"].values)
    X_word_matrix = tokenizer.texts_to_matrix(X["name"].values, mode="count")
    
    y = df["success"]
    
    return X, X_word_matrix, y

ModuleNotFoundError: No module named 'tensorflow'

In [46]:
train_full = pd.read_csv("data/2018-train.csv").dropna()
validate_full = pd.read_csv("data/2018-validate.csv").dropna()

In [47]:
train_pp = preprocess_df(train_full)
validate_pp = preprocess_df(validate_full)

In [48]:
train_clean = clean_df(train_pp)
validate_clean = clean_df(validate_pp)

In [49]:
train_clean.head(2)

,Unnamed: 0,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,success,duration
0,319122,695425648,Peace-building through story-making with youth...,Children's Books,Publishing,SEK,2015-06-09,6000.0,2015-05-10 14:53:53,6251.0,successful,15,SE,756.92,762.02,731.42,True,29 days 09:06:07
2,175494,189251239,Colored Baggies for Boardgames,Tabletop Games,Games,USD,2013-01-07,6000.0,2012-11-08 20:06:31,15151.0,successful,518,US,15151.00,15151.00,6000.00,True,59 days 03:53:29


In [50]:
train_clean.loc[train_clean["pledged"] >= train_clean["goal"]].groupby("state").count()

,Unnamed: 0,ID,name,category,main_category,currency,deadline,goal,launched,pledged,backers,country,usd pledged,usd_pledged_real,usd_goal_real,success,duration
state,,,,,,,,,,,,,,,,,
failed,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
successful,100364,100364,100364,100364,100364,100364,100364,100364,100364,100364,100364,100364,100364,100364,100364,100364,100364
suspended,228,228,228,228,228,228,228,228,228,228,228,228,228,228,228,228,228


In [51]:
train_X, train_X_word_matrix train_y = get_X_X_word_matrix_y(train_clean)

In [52]:
train_X.head(2)

,name,duration,usd_goal_real,Art,Comics,Crafts,Dance,Design,Fashion,Film & Video,...,Wearables,Weaving,Web,Webcomics,Webseries,Woodworking,Workshops,World Music,Young Adult,Zines
0,Peace-building through story-making with youth...,32767,731.42,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Colored Baggies for Boardgames,14009,6000.00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1]:
# https://stackoverflow.com/questions/49175961/how-to-use-additional-features-along-with-word-embeddings-in-keras
# https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
# https://machinelearningmastery.com/prepare-text-data-deep-learning-keras/
# https://www.researchgate.net/post/Why_we_use_keras_tokenizer_before_embedding_layer_in_Deep_learning_for_text_data
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate, Dense, Activation, LSTM

embed = Embedding(1000, 100)
lstm = LSTM(100)(embed)
other_input = Input(shape=(len(train_X.columns),))
conc = Concatenate()(lstm, other_input)
dens = Dense(units=30)(conc)
acti = Activation("sigmoid")(dens)
model = Model([embed, other_input], acti)
model.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])


ModuleNotFoundError: No module named 'tensorflow'

In [39]:
from tensorflow.keras.layers import Dense

model.add(Dense(units=30, activation="relu", input_dim=len(train_X.columns)))
model.add(Dense(units=30, activation="relu", input_dim=30))
model.add(Dense(units=1, activation="softmax"))

In [40]:
model.compile(loss="mean_squared_error",
             optimizer = "sgd",
             metrics=["accuracy"])

In [41]:
model.fit(train_X, train_y, epochs=5, batch_size=32)

Train on 249956 samples
Epoch 1/5
249956/249956 [==============================] - 15s 61us/sample - loss: 0.5976 - accuracy: 0.4024
Epoch 2/5
249956/249956 [==============================] - 15s 60us/sample - loss: 0.5976 - accuracy: 0.4024
Epoch 3/5
249956/249956 [==============================] - 15s 61us/sample - loss: 0.5976 - accuracy: 0.4024
Epoch 4/5
249956/249956 [==============================] - 15s 60us/sample - loss: 0.5976 - accuracy: 0.4024
Epoch 5/5
249956/249956 [==============================] - 15s 61us/sample - loss: 0.5976 - accuracy: 0.4024


In [42]:
val_X, val_y = get_X_y(validate_clean)

In [43]:
model.evaluate(val_X, val_y)

33249/33249 [==============================] - 1s 33us/sample - loss: 0.6008 - accuracy: 0.3992


[0.600800024060874, 0.39919996]